In [20]:
# Core Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings

# Scikit-learn for preprocessing and modeling
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer # Using SimpleImputer instead of KNN/XGB for EXT_SOURCE initially for simplicity, can swap later
from sklearn.preprocessing import OneHotEncoder # For Bureau categoricals
from xgboost import XGBRegressor # Keep for EXT_SOURCE imputation as in original code

# Display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.5f' % x)
pd.set_option('future.no_silent_downcasting', True) # Future warning for downcasting, can be useful to catch potential issues
warnings.filterwarnings('ignore', category=UserWarning) # Suppress SettingWithCopyWarning for cleaner output, use .copy() where needed

In [21]:
# --- 1. Loading Functions ---

def load_initial_data(data_path):
    """Loads the required raw data files."""
    print("Loading raw data...")
    try:
        app_train_df = pd.read_csv(data_path + 'application_train.csv')
        bureau_df = pd.read_csv(data_path + 'bureau.csv')
        bureau_balance_df = pd.read_csv(data_path + 'bureau_balance.csv')
        
        # Ensure bureau_df only contains SK_ID_CURR in app_train_df
        bureau_df = bureau_df[bureau_df['SK_ID_CURR'].isin(app_train_df['SK_ID_CURR'])]
        bureau_balance_df = bureau_balance_df[bureau_balance_df['SK_ID_BUREAU'].isin(bureau_df['SK_ID_BUREAU'])]
        
        print("Raw data loaded successfully.")
        return app_train_df, bureau_df, bureau_balance_df
    except FileNotFoundError as e:
        print(f"Error loading files: {e}. Make sure files are in {data_path}")
        raise

In [22]:
# --- 2. Application Data Preprocessing & Feature Engineering Functions ---

def split_application_data(app_df, target_col='TARGET', test_size=0.2, random_state=42):
    """Splits the application data into training and testing sets."""
    print(f"Splitting application data (Test size: {test_size}, Random State: {random_state})...")
    X = app_df.drop(columns=[target_col])
    y = app_df[target_col]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=y)
    print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")
    return X_train, X_test, y_train, y_test

def clean_application(df):
    """Applies basic cleaning steps to application data (train or test)."""
    df_cleaned = df.copy()
    print(f"Applying basic cleaning to DataFrame with shape: {df_cleaned.shape}")

    # Drop specified FLAG_DOCUMENT columns (low variance)
    flag_cols_to_drop = ['FLAG_DOCUMENT_2', 'FLAG_DOCUMENT_4', 'FLAG_DOCUMENT_10', 'FLAG_DOCUMENT_12', 'FLAG_DOCUMENT_20']
    # Ensure columns exist before dropping
    flag_cols_to_drop = [col for col in flag_cols_to_drop if col in df_cleaned.columns]
    df_cleaned = df_cleaned.drop(flag_cols_to_drop, axis=1)
    print(f"Dropped low-variance FLAG columns: {flag_cols_to_drop}")

    # Convert DAYS_BIRTH to years (positive)
    if 'DAYS_BIRTH' in df_cleaned.columns:
        df_cleaned['DAYS_BIRTH'] = df_cleaned['DAYS_BIRTH'] * -1 / 365
        print("Converted DAYS_BIRTH to years.")

    # Handle DAYS_EMPLOYED anomaly (365243 -> NaN)
    if 'DAYS_EMPLOYED' in df_cleaned.columns:
        df_cleaned['DAYS_EMPLOYED'] = df_cleaned['DAYS_EMPLOYED'].replace(365243, np.nan)
        print("Replaced DAYS_EMPLOYED anomaly (365243) with NaN.")

    # Handle OBS_..._SOCIAL_CIRCLE anomalies (> 30 -> NaN) - Corrected assignment
    # Ensure these columns exist before attempting replacement
    if 'OBS_30_CNT_SOCIAL_CIRCLE' in df_cleaned.columns:
        df_cleaned.loc[df_cleaned['OBS_30_CNT_SOCIAL_CIRCLE'] > 30, 'OBS_30_CNT_SOCIAL_CIRCLE'] = np.nan
        print("Capped OBS_30_CNT_SOCIAL_CIRCLE at 30 (values > 30 set to NaN).")
    if 'OBS_60_CNT_SOCIAL_CIRCLE' in df_cleaned.columns:
        df_cleaned.loc[df_cleaned['OBS_60_CNT_SOCIAL_CIRCLE'] > 30, 'OBS_60_CNT_SOCIAL_CIRCLE'] = np.nan
        print("Capped OBS_60_CNT_SOCIAL_CIRCLE at 30 (values > 30 set to NaN).")

    # Note: Removing 'XNA' Gender was in the original code.
    # It's generally better to handle this *before* splitting if it's a clear data error,
    # or handle it as another category ('XNA') during encoding if it might be meaningful.
    # We'll keep the original behavior but apply it *before* the split for simplicity here.
    # (See `preprocess_credit_data` function where this is called before split)

    # Convert REGION_RATING columns to object type (as per original logic, for later encoding)
    # This assumes they will be treated as categorical later (e.g., by Response Encoding)
    if 'REGION_RATING_CLIENT' in df_cleaned.columns:
        df_cleaned['REGION_RATING_CLIENT'] = df_cleaned['REGION_RATING_CLIENT'].astype('object')
        print("Converted REGION_RATING_CLIENT to object type.")
    if 'REGION_RATING_CLIENT_W_CITY' in df_cleaned.columns:
        df_cleaned['REGION_RATING_CLIENT_W_CITY'] = df_cleaned['REGION_RATING_CLIENT_W_CITY'].astype('object')
        print("Converted REGION_RATING_CLIENT_W_CITY to object type.")

    print("Basic cleaning finished.")
    return df_cleaned

def handle_app_missing_categorical(X_train, X_test):
    """Fills missing values in categorical columns with 'XNA'."""
    print("Handling missing categorical values (filling with 'XNA')...")
    X_train_filled = X_train.copy()
    X_test_filled = X_test.copy()

    categorical_cols = X_train_filled.select_dtypes(include='object').columns.tolist()

    X_train_filled[categorical_cols] = X_train_filled[categorical_cols].fillna('XNA')
    X_test_filled[categorical_cols] = X_test_filled[categorical_cols].fillna('XNA')

    print(f"Filled NaNs in {len(categorical_cols)} categorical columns.")
    return X_train_filled, X_test_filled

def impute_ext_sources_xgb(X_train, X_test):
    """Imputes missing EXT_SOURCE features using XGBoost Regressor.
       Fits on train data, transforms both train and test."""
    print("Imputing EXT_SOURCE columns using XGBoost...")
    X_train_imputed = X_train.copy()
    X_test_imputed = X_test.copy()

    # Identify numeric columns available for prediction (excluding target and IDs)
    numeric_cols = X_train_imputed.select_dtypes(include=np.number).columns.tolist()
    exclude_cols = ['SK_ID_CURR', 'TARGET', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']
    cols_for_modelling_base = [col for col in numeric_cols if col not in exclude_cols]

    # Impute in order of least missing to most missing
    for ext_col in ['EXT_SOURCE_2', 'EXT_SOURCE_3', 'EXT_SOURCE_1']:
        if ext_col not in X_train_imputed.columns:
            print(f"Column {ext_col} not found, skipping imputation.")
            continue

        print(f"Imputing {ext_col}...")
        # Define current set of predictor columns
        cols_for_modelling = [col for col in cols_for_modelling_base if col in X_train_imputed.columns]
        # Add previously imputed EXT_SOURCE cols if they exist
        prev_ext_cols = [c for c in ['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3'] if c != ext_col and c in X_train_imputed.columns and X_train_imputed[c].isnull().sum() == 0]
        cols_for_modelling.extend(prev_ext_cols)
        cols_for_modelling = list(set(cols_for_modelling)) # Ensure uniqueness

        # Prepare data for this specific column imputation
        train_not_missing = X_train_imputed[X_train_imputed[ext_col].notna()]
        train_missing = X_train_imputed[X_train_imputed[ext_col].isna()]
        test_missing = X_test_imputed[X_test_imputed[ext_col].isna()]

        if train_missing.empty and test_missing.empty:
             print(f"No missing values found for {ext_col}. Skipping.")
             continue

        # Only proceed if there are non-missing values to train on
        if not train_not_missing.empty:
            X_model_train = train_not_missing[cols_for_modelling].fillna(train_not_missing[cols_for_modelling].median()) # Handle potential NaNs in predictors
            Y_model_train = train_not_missing[ext_col]

            # Define and train the model
            xg = XGBRegressor(n_estimators=100, max_depth=3, learning_rate=0.1, n_jobs=-1, random_state=59, objective='reg:squarederror')
            xg.fit(X_model_train, Y_model_train)
            print(f"Trained XGBoost model for {ext_col}.")

            # Impute missing values in train set
            if not train_missing.empty:
                X_pred_train = train_missing[cols_for_modelling].fillna(train_not_missing[cols_for_modelling].median()) # Use median from non-missing train part
                X_train_imputed.loc[X_train_imputed[ext_col].isna(), ext_col] = xg.predict(X_pred_train)
                print(f"Imputed {train_missing.shape[0]} values in train set for {ext_col}.")

            # Impute missing values in test set
            if not test_missing.empty:
                X_pred_test = test_missing[cols_for_modelling].fillna(train_not_missing[cols_for_modelling].median()) # Use median from non-missing train part
                X_test_imputed.loc[X_test_imputed[ext_col].isna(), ext_col] = xg.predict(X_pred_test)
                print(f"Imputed {test_missing.shape[0]} values in test set for {ext_col}.")
        else:
             print(f"Warning: No non-missing values found for {ext_col} in training data. Cannot train imputer. Filling with global median.")
             global_median = X_train_imputed[ext_col].median() # Fallback: should not happen if called in order
             X_train_imputed[ext_col] = X_train_imputed[ext_col].fillna(global_median)
             X_test_imputed[ext_col] = X_test_imputed[ext_col].fillna(global_median)


    print("Finished EXT_SOURCE imputation.")
    return X_train_imputed, X_test_imputed

def engineer_app_numeric_features(df):
    """Engineers numeric features based on domain knowledge for application data."""
    eng_df = df.copy()
    print(f"Applying numeric feature engineering to DataFrame with shape: {eng_df.shape}")

    # Convert REGION_RATING back to numeric for calculations if they exist and are object type
    if 'REGION_RATING_CLIENT' in eng_df.columns and eng_df['REGION_RATING_CLIENT'].dtype == 'object':
        # Attempt conversion, coercing errors to NaN (which might be handled later or indicate issues)
        eng_df['REGION_RATING_CLIENT'] = pd.to_numeric(eng_df['REGION_RATING_CLIENT'], errors='coerce')
    if 'REGION_RATING_CLIENT_W_CITY' in eng_df.columns and eng_df['REGION_RATING_CLIENT_W_CITY'].dtype == 'object':
        eng_df['REGION_RATING_CLIENT_W_CITY'] = pd.to_numeric(eng_df['REGION_RATING_CLIENT_W_CITY'], errors='coerce')

    # Define epsilon for safe division
    eps = 1e-5

    # Income / Credit Ratios
    # Ensure required columns exist before creating features
    if 'AMT_CREDIT' in eng_df.columns and 'AMT_INCOME_TOTAL' in eng_df.columns:
        eng_df['CREDIT_INCOME_RATIO'] = eng_df['AMT_CREDIT'] / (eng_df['AMT_INCOME_TOTAL'] + eps)
    if 'AMT_CREDIT' in eng_df.columns and 'AMT_ANNUITY' in eng_df.columns:
        eng_df['CREDIT_ANNUITY_RATIO'] = eng_df['AMT_CREDIT'] / (eng_df['AMT_ANNUITY'] + eps)
    if 'AMT_ANNUITY' in eng_df.columns and 'AMT_INCOME_TOTAL' in eng_df.columns:
        eng_df['ANNUITY_INCOME_RATIO'] = eng_df['AMT_ANNUITY'] / (eng_df['AMT_INCOME_TOTAL'] + eps)
        eng_df['INCOME_ANNUITY_DIFF'] = eng_df['AMT_INCOME_TOTAL'] - eng_df['AMT_ANNUITY']
    if 'AMT_CREDIT' in eng_df.columns and 'AMT_GOODS_PRICE' in eng_df.columns:
        eng_df['CREDIT_GOODS_RATIO'] = eng_df['AMT_CREDIT'] / (eng_df['AMT_GOODS_PRICE'] + eps)
        eng_df['CREDIT_GOODS_DIFF'] = eng_df['AMT_CREDIT'] - eng_df['AMT_GOODS_PRICE']
    if 'AMT_GOODS_PRICE' in eng_df.columns and 'AMT_INCOME_TOTAL' in eng_df.columns:
        eng_df['GOODS_INCOME_RATIO'] = eng_df['AMT_GOODS_PRICE'] / (eng_df['AMT_INCOME_TOTAL'] + eps)
    if 'AMT_INCOME_TOTAL' in eng_df.columns and 'EXT_SOURCE_3' in eng_df.columns:
         eng_df['INCOME_EXT_RATIO'] = eng_df['AMT_INCOME_TOTAL'] / (eng_df['EXT_SOURCE_3'] + eps)
    if 'AMT_CREDIT' in eng_df.columns and 'EXT_SOURCE_3' in eng_df.columns:
         eng_df['CREDIT_EXT_RATIO'] = eng_df['AMT_CREDIT'] / (eng_df['EXT_SOURCE_3'] + eps)

    # Age / Employment Ratios
    if 'DAYS_BIRTH' in eng_df.columns and 'DAYS_EMPLOYED' in eng_df.columns:
        # Note: DAYS_BIRTH is now positive years. DAYS_EMPLOYED might be negative days or NaN.
        # To make comparable, convert DAYS_EMPLOYED to positive years if not NaN.
        days_employed_years = eng_df['DAYS_EMPLOYED'].apply(lambda x: -x / 365 if pd.notna(x) else np.nan)
        eng_df['AGE_EMPLOYED_DIFF'] = eng_df['DAYS_BIRTH'] - days_employed_years # Diff in years
        eng_df['EMPLOYED_TO_AGE_RATIO'] = days_employed_years / (eng_df['DAYS_BIRTH'] + eps)

    # Car Ratios
    if 'OWN_CAR_AGE' in eng_df.columns and 'DAYS_EMPLOYED' in eng_df.columns:
        days_employed_years = eng_df['DAYS_EMPLOYED'].apply(lambda x: -x / 365 if pd.notna(x) else np.nan)
        eng_df['CAR_EMPLOYED_DIFF'] = eng_df['OWN_CAR_AGE'] - days_employed_years
        eng_df['CAR_EMPLOYED_RATIO'] = eng_df['OWN_CAR_AGE'] / (days_employed_years + eps)
    if 'DAYS_BIRTH' in eng_df.columns and 'OWN_CAR_AGE' in eng_df.columns:
        eng_df['CAR_AGE_DIFF'] = eng_df['DAYS_BIRTH'] - eng_df['OWN_CAR_AGE']
        eng_df['CAR_AGE_RATIO'] = eng_df['OWN_CAR_AGE'] / (eng_df['DAYS_BIRTH'] + eps)

    # Flag Contacts Sum
    contact_flags = ['FLAG_MOBIL', 'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE', 'FLAG_PHONE', 'FLAG_EMAIL']
    existing_contact_flags = [flag for flag in contact_flags if flag in eng_df.columns]
    if existing_contact_flags:
        eng_df['FLAG_CONTACTS_SUM'] = eng_df[existing_contact_flags].sum(axis=1)

    # Hour Process * Credit
    if 'AMT_CREDIT' in eng_df.columns and 'HOUR_APPR_PROCESS_START' in eng_df.columns:
        eng_df['HOUR_PROCESS_CREDIT_MUL'] = eng_df['AMT_CREDIT'] * eng_df['HOUR_APPR_PROCESS_START']

    # Family Members
    if 'CNT_FAM_MEMBERS' in eng_df.columns and 'CNT_CHILDREN' in eng_df.columns:
        eng_df['CNT_NON_CHILDREN'] = eng_df['CNT_FAM_MEMBERS'] - eng_df['CNT_CHILDREN']
    if 'CNT_CHILDREN' in eng_df.columns and 'AMT_INCOME_TOTAL' in eng_df.columns:
        eng_df['CHILDREN_INCOME_RATIO'] = eng_df['CNT_CHILDREN'] / (eng_df['AMT_INCOME_TOTAL'] + eps)
    if 'AMT_INCOME_TOTAL' in eng_df.columns and 'CNT_FAM_MEMBERS' in eng_df.columns:
        eng_df['PER_CAPITA_INCOME'] = eng_df['AMT_INCOME_TOTAL'] / (eng_df['CNT_FAM_MEMBERS'] + 1) # Add 1 to avoid division by zero if CNT_FAM_MEMBERS is 0

    # Region Ratings (assuming they are numeric now)
    if 'REGION_RATING_CLIENT' in eng_df.columns and 'REGION_RATING_CLIENT_W_CITY' in eng_df.columns and 'AMT_INCOME_TOTAL' in eng_df.columns:
         eng_df['REGIONS_RATING_INCOME_MUL'] = (eng_df['REGION_RATING_CLIENT'] + eng_df['REGION_RATING_CLIENT_W_CITY']) * eng_df['AMT_INCOME_TOTAL'] / 2
         # Use .max(axis=1) and .min(axis=1) for row-wise operations
         eng_df['REGION_RATING_MAX'] = eng_df[['REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY']].max(axis=1)
         # Original code had a typo assigning min to max. Corrected:
         eng_df['REGION_RATING_MIN'] = eng_df[['REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY']].min(axis=1)
         eng_df['REGION_RATING_MEAN'] = eng_df[['REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY']].mean(axis=1)
         eng_df['REGION_RATING_MUL'] = eng_df['REGION_RATING_CLIENT'] * eng_df['REGION_RATING_CLIENT_W_CITY']

    # Flag Regions Sum
    region_flags = ['REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION', 'LIVE_REGION_NOT_WORK_REGION',
                    'REG_CITY_NOT_LIVE_CITY', 'REG_CITY_NOT_WORK_CITY', 'LIVE_CITY_NOT_WORK_CITY']
    existing_region_flags = [flag for flag in region_flags if flag in eng_df.columns]
    if existing_region_flags:
        eng_df['FLAG_REGIONS_SUM'] = eng_df[existing_region_flags].sum(axis=1)

    # External Sources Aggregations
    ext_source_cols = ['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']
    existing_ext_sources = [col for col in ext_source_cols if col in eng_df.columns]
    if len(existing_ext_sources) > 0:
        eng_df['EXT_SOURCE_MEAN'] = eng_df[existing_ext_sources].mean(axis=1)
        eng_df['EXT_SOURCE_STD'] = eng_df[existing_ext_sources].std(axis=1)
        eng_df['EXT_SOURCE_MAX'] = eng_df[existing_ext_sources].max(axis=1)
        eng_df['EXT_SOURCE_MIN'] = eng_df[existing_ext_sources].min(axis=1)
        # Original multiplication might lead to very small/large numbers or zero if one source is missing/zero
        # eng_df['EXT_SOURCE_MUL'] = eng_df[existing_ext_sources].prod(axis=1) # Use prod for multiplication
        # Weighted sum might be more robust
        weights = {'EXT_SOURCE_1': 2, 'EXT_SOURCE_2': 3, 'EXT_SOURCE_3': 4}
        weighted_sum = pd.Series(0, index=eng_df.index, dtype=float)
        for col in existing_ext_sources:
            weighted_sum += eng_df[col].fillna(0) * weights.get(col, 1) # Fill NaN with 0 for sum
        eng_df['WEIGHTED_EXT_SOURCE'] = weighted_sum


    # Apartment Scores Aggregations
    # Define groups of apartment features
    apartment_groups = {
        'AVG': ['APARTMENTS_AVG', 'BASEMENTAREA_AVG', 'YEARS_BEGINEXPLUATATION_AVG', 'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'ELEVATORS_AVG', 'ENTRANCES_AVG', 'FLOORSMAX_AVG', 'FLOORSMIN_AVG', 'LANDAREA_AVG', 'LIVINGAPARTMENTS_AVG', 'LIVINGAREA_AVG', 'NONLIVINGAPARTMENTS_AVG', 'NONLIVINGAREA_AVG'],
        'MODE': ['APARTMENTS_MODE', 'BASEMENTAREA_MODE', 'YEARS_BEGINEXPLUATATION_MODE', 'YEARS_BUILD_MODE', 'COMMONAREA_MODE', 'ELEVATORS_MODE', 'ENTRANCES_MODE', 'FLOORSMAX_MODE', 'FLOORSMIN_MODE', 'LANDAREA_MODE', 'LIVINGAPARTMENTS_MODE', 'LIVINGAREA_MODE', 'NONLIVINGAPARTMENTS_MODE', 'NONLIVINGAREA_MODE', 'TOTALAREA_MODE'],
        'MEDI': ['APARTMENTS_MEDI', 'BASEMENTAREA_MEDI', 'YEARS_BEGINEXPLUATATION_MEDI', 'YEARS_BUILD_MEDI', 'COMMONAREA_MEDI', 'ELEVATORS_MEDI', 'ENTRANCES_MEDI', 'FLOORSMAX_MEDI', 'FLOORSMIN_MEDI', 'LANDAREA_MEDI', 'LIVINGAPARTMENTS_MEDI', 'LIVINGAREA_MEDI', 'NONLIVINGAPARTMENTS_MEDI', 'NONLIVINGAREA_MEDI']
    }
    for suffix, cols in apartment_groups.items():
        existing_cols = [col for col in cols if col in eng_df.columns]
        if existing_cols:
            eng_df[f'APARTMENTS_SUM_{suffix}'] = eng_df[existing_cols].sum(axis=1)
            if f'APARTMENTS_SUM_{suffix}' in eng_df.columns and 'AMT_INCOME_TOTAL' in eng_df.columns:
                eng_df[f'INCOME_APARTMENT_{suffix}_MUL'] = eng_df[f'APARTMENTS_SUM_{suffix}'] * eng_df['AMT_INCOME_TOTAL']


    # OBS and DEF Social Circle Features
    if 'OBS_30_CNT_SOCIAL_CIRCLE' in eng_df.columns and 'OBS_60_CNT_SOCIAL_CIRCLE' in eng_df.columns:
        eng_df['OBS_30_60_SUM'] = eng_df['OBS_30_CNT_SOCIAL_CIRCLE'] + eng_df['OBS_60_CNT_SOCIAL_CIRCLE']
    if 'DEF_30_CNT_SOCIAL_CIRCLE' in eng_df.columns and 'DEF_60_CNT_SOCIAL_CIRCLE' in eng_df.columns:
        eng_df['DEF_30_60_SUM'] = eng_df['DEF_30_CNT_SOCIAL_CIRCLE'] + eng_df['DEF_60_CNT_SOCIAL_CIRCLE']
    if 'OBS_30_CNT_SOCIAL_CIRCLE' in eng_df.columns and 'DEF_30_CNT_SOCIAL_CIRCLE' in eng_df.columns:
        eng_df['OBS_DEF_30_MUL'] = eng_df['OBS_30_CNT_SOCIAL_CIRCLE'] * eng_df['DEF_30_CNT_SOCIAL_CIRCLE']
    if 'OBS_60_CNT_SOCIAL_CIRCLE' in eng_df.columns and 'DEF_60_CNT_SOCIAL_CIRCLE' in eng_df.columns:
        eng_df['OBS_DEF_60_MUL'] = eng_df['OBS_60_CNT_SOCIAL_CIRCLE'] * eng_df['DEF_60_CNT_SOCIAL_CIRCLE']
    obs_def_cols = ['OBS_30_CNT_SOCIAL_CIRCLE', 'DEF_30_CNT_SOCIAL_CIRCLE', 'OBS_60_CNT_SOCIAL_CIRCLE', 'DEF_60_CNT_SOCIAL_CIRCLE']
    existing_obs_def = [col for col in obs_def_cols if col in eng_df.columns]
    if existing_obs_def:
        eng_df['SUM_OBS_DEF_ALL'] = eng_df[existing_obs_def].sum(axis=1)
    if 'AMT_CREDIT' in eng_df.columns:
        if 'OBS_30_CNT_SOCIAL_CIRCLE' in eng_df.columns:
             eng_df['OBS_30_CREDIT_RATIO'] = eng_df['AMT_CREDIT'] / (eng_df['OBS_30_CNT_SOCIAL_CIRCLE'] + eps)
        if 'OBS_60_CNT_SOCIAL_CIRCLE' in eng_df.columns:
             eng_df['OBS_60_CREDIT_RATIO'] = eng_df['AMT_CREDIT'] / (eng_df['OBS_60_CNT_SOCIAL_CIRCLE'] + eps)
        if 'DEF_30_CNT_SOCIAL_CIRCLE' in eng_df.columns:
             eng_df['DEF_30_CREDIT_RATIO'] = eng_df['AMT_CREDIT'] / (eng_df['DEF_30_CNT_SOCIAL_CIRCLE'] + eps)
        if 'DEF_60_CNT_SOCIAL_CIRCLE' in eng_df.columns:
             eng_df['DEF_60_CREDIT_RATIO'] = eng_df['AMT_CREDIT'] / (eng_df['DEF_60_CNT_SOCIAL_CIRCLE'] + eps)

    # Flag Documents Sum (excluding those dropped earlier)
    flag_cols_to_drop = ['FLAG_DOCUMENT_2', 'FLAG_DOCUMENT_4', 'FLAG_DOCUMENT_10', 'FLAG_DOCUMENT_12', 'FLAG_DOCUMENT_20']
    doc_flags = [f'FLAG_DOCUMENT_{i}' for i in range(3, 22) if f'FLAG_DOCUMENT_{i}' not in flag_cols_to_drop] # Original range was up to 21
    existing_doc_flags = [flag for flag in doc_flags if flag in eng_df.columns]
    if existing_doc_flags:
        eng_df['SUM_FLAGS_DOCUMENTS'] = eng_df[existing_doc_flags].sum(axis=1)

    # Details Change Features
    days_change_cols = ['DAYS_LAST_PHONE_CHANGE', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH']
    existing_days_change = [col for col in days_change_cols if col in eng_df.columns]
    if len(existing_days_change) > 0:
        # Multiplication might be sensitive to large negative numbers. Sum is safer.
        # eng_df['DAYS_DETAILS_CHANGE_MUL'] = eng_df[existing_days_change].prod(axis=1)
        eng_df['DAYS_DETAILS_CHANGE_SUM'] = eng_df[existing_days_change].sum(axis=1)


    # Enquiries Sum and Ratio
    enq_cols = ['AMT_REQ_CREDIT_BUREAU_HOUR', 'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK',
                'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT', 'AMT_REQ_CREDIT_BUREAU_YEAR']
    existing_enq_cols = [col for col in enq_cols if col in eng_df.columns]
    if existing_enq_cols:
        eng_df['AMT_ENQ_SUM'] = eng_df[existing_enq_cols].sum(axis=1)
        if 'AMT_CREDIT' in eng_df.columns and 'AMT_ENQ_SUM' in eng_df.columns:
             eng_df['ENQ_CREDIT_RATIO'] = eng_df['AMT_ENQ_SUM'] / (eng_df['AMT_CREDIT'] + eps)

    # Feature: Total missing values count
    eng_df['MISSING_VALS_TOTAL_APP'] = eng_df.isnull().sum(axis=1)

    print(f"Numeric feature engineering finished. New shape: {eng_df.shape}")
    return eng_df

def engineer_app_categorical_interactions(X_train, X_test, y_train):
    """Generates features based on categorical groupings and aggregates.
       Fits on train, transforms both train and test."""
    print("Engineering categorical interaction features...")
    X_train_eng = X_train.copy()
    X_test_eng = X_test.copy()
    # Temporarily add target back to X_train for fitting aggregations
    X_train_eng['TARGET'] = y_train

    # Define groups and aggregations (as per original code)
    columns_to_aggregate_on = [
        ['NAME_CONTRACT_TYPE', 'NAME_INCOME_TYPE', 'OCCUPATION_TYPE'],
        ['CODE_GENDER', 'NAME_FAMILY_STATUS', 'NAME_INCOME_TYPE'],
        ['FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NAME_INCOME_TYPE'],
        ['NAME_EDUCATION_TYPE', 'NAME_INCOME_TYPE', 'OCCUPATION_TYPE'],
        ['OCCUPATION_TYPE', 'ORGANIZATION_TYPE'],
        ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']
    ]
    aggregations = {
        'AMT_ANNUITY': ['mean', 'max', 'min'],
        'ANNUITY_INCOME_RATIO': ['mean', 'max', 'min'],
        'AGE_EMPLOYED_DIFF': ['mean', 'min'],
        'AMT_INCOME_TOTAL': ['mean', 'max', 'min'],
        # 'APARTMENTS_SUM_AVG': ['mean', 'max', 'min'], # These were created in numeric FE
        # 'APARTMENTS_SUM_MEDI': ['mean', 'max', 'min'],# These were created in numeric FE
        'EXT_SOURCE_MEAN': ['mean', 'max', 'min'],
        'EXT_SOURCE_1': ['mean', 'max', 'min'],
        'EXT_SOURCE_2': ['mean', 'max', 'min'],
        'EXT_SOURCE_3': ['mean', 'max', 'min'],
        # Add target mean for the group (common technique)
        'TARGET': ['mean']
    }

    # Filter groups and aggregations based on available columns
    valid_groups = []
    for group in columns_to_aggregate_on:
        if all(col in X_train_eng.columns for col in group):
            valid_groups.append(group)
        else:
            print(f"Skipping group due to missing columns: {group}")

    valid_aggregations = {}
    for agg_col, agg_funcs in aggregations.items():
         if agg_col in X_train_eng.columns:
             valid_aggregations[agg_col] = agg_funcs
         else:
             print(f"Skipping aggregation on missing column: {agg_col}")


    if not valid_aggregations:
         print("No valid columns found for categorical interaction aggregations. Skipping.")
         # Drop target added temporarily
         X_train_eng.drop(columns=['TARGET'], inplace=True)
         return X_train_eng, X_test_eng

    # Apply aggregations
    for group in valid_groups:
        print(f"Aggregating on group: {group}")
        # Ensure group columns are hashable (e.g. no lists within cells)
        try:
            grouped_interactions = X_train_eng.groupby(group).agg(valid_aggregations)
            new_colnames = ['_'.join(ele).upper() + '_AGG_' + '_'.join(group) for ele in grouped_interactions.columns]
            grouped_interactions.columns = new_colnames

            # Merge onto train and test
            X_train_eng = X_train_eng.merge(grouped_interactions, on=group, how='left')
            X_test_eng = X_test_eng.merge(grouped_interactions, on=group, how='left')
            print(f"Merged {len(new_colnames)} features for group {group}.")

        except Exception as e:
            print(f"Could not aggregate on group {group}. Error: {e}")


    # Drop target added temporarily
    X_train_eng = X_train_eng.drop(columns=['TARGET'])

    print("Finished categorical interaction features.")
    return X_train_eng, X_test_eng

def encode_app_categorical_response(X_train, X_test, y_train):
    """Encodes categorical features using Response Encoding.
       Fits on train, transforms both train and test."""
    print("Applying Response Encoding...")
    X_train_encoded = X_train.copy()
    X_test_encoded = X_test.copy()
    X_train_encoded['TARGET'] = y_train # Temporarily add target for fitting

    categorical_cols = X_train_encoded.select_dtypes(include='object').columns.tolist()
    original_cols_to_drop = []

    for col in categorical_cols:
        print(f"Encoding column: {col}")
        # Calculate conditional probabilities P(Category|Target=1) and P(Category|Target=0)
        # Smoothed mean: (mean * count + global_mean * alpha) / (count + alpha)
        global_mean = X_train_encoded['TARGET'].mean()
        agg = X_train_encoded.groupby(col)['TARGET'].agg(['mean', 'count'])
        counts = agg['count']
        means = agg['mean']
        alpha = 10 # Smoothing factor

        smoothed_mean = (means * counts + global_mean * alpha) / (counts + alpha)

        # Create mapping dictionary (for probability of Target=1)
        # For Target=0, it's simply 1 - P(Target=1) conceptually,
        # but the original code created two columns based on P(category | label).
        # Let's simplify to P(Target=1 | category) which is common target encoding.
        # We can add P(Target=0 | category) if needed, it's just 1 minus the first.
        mapping_dict_1 = smoothed_mean.to_dict()

        # Transform train and test
        X_train_encoded[col + '_RESPONSE_1'] = X_train_encoded[col].map(mapping_dict_1)
        X_test_encoded[col + '_RESPONSE_1'] = X_test_encoded[col].map(mapping_dict_1)

        # Fill potential NaNs in test set (categories not seen in train) with global mean
        X_test_encoded[col + '_RESPONSE_1'] = X_test_encoded[col + '_RESPONSE_1'].fillna(global_mean)

        # Add P(Target=0 | category) column
        # X_train_encoded[col + '_RESPONSE_0'] = 1 - X_train_encoded[col + '_RESPONSE_1']
        # X_test_encoded[col + '_RESPONSE_0'] = 1 - X_test_encoded[col + '_RESPONSE_1']

        original_cols_to_drop.append(col)

    # Drop original categorical columns and the temporary target column
    X_train_encoded = X_train_encoded.drop(columns=original_cols_to_drop + ['TARGET'])
    X_test_encoded = X_test_encoded.drop(columns=original_cols_to_drop)

    print(f"Response Encoding finished. Dropped original columns: {original_cols_to_drop}")
    return X_train_encoded, X_test_encoded

In [23]:
# --- 3. Bureau & Bureau Balance Preprocessing Functions ---

def preprocess_bureau_balance(bureau_balance_df):
    """Preprocesses bureau_balance: encodes status, creates weighted/EWM features, aggregates by SK_ID_BUREAU."""
    print("Preprocessing bureau_balance...")
    bb = bureau_balance_df.copy()
    initial_shape = bb.shape

    # Encode STATUS
    # Original mapping: {'C': 0, '0': 1, '1': 2, '2': 3, 'X': 4, '3': 5, '4': 6, '5': 7}
    # Simpler numeric mapping might also work, but let's keep the original logic
    dict_for_status = {'C': 0, '0': 1, '1': 2, '2': 3, 'X': 4, '3': 5, '4': 6, '5': 7}
    bb['STATUS_ENCODED'] = bb['STATUS'].map(dict_for_status)

    # Weighted status
    bb['MONTHS_BALANCE_POS'] = np.abs(bb['MONTHS_BALANCE'])
    bb['WEIGHTED_STATUS'] = bb['STATUS_ENCODED'] / (bb['MONTHS_BALANCE_POS'] + 1)

    # Sort for EWM calculation
    bb = bb.sort_values(by=['SK_ID_BUREAU', 'MONTHS_BALANCE_POS'], ascending=[True, True]) # Ascending months

    # Exponential Weighted Moving Average (EWMA)
    alpha = 0.8
    bb['EXP_WEIGHTED_STATUS'] = bb.groupby('SK_ID_BUREAU')['WEIGHTED_STATUS'].transform(lambda x: x.ewm(alpha=alpha).mean())
    bb['EXP_ENCODED_STATUS'] = bb.groupby('SK_ID_BUREAU')['STATUS_ENCODED'].transform(lambda x: x.ewm(alpha=alpha).mean())

    # Yearly Aggregations
    bb['YEAR_BALANCE'] = bb['MONTHS_BALANCE_POS'] // 12

    # Define aggregations
    aggregations_basic = {
        'MONTHS_BALANCE_POS': ['count', 'mean', 'max'], # Added count
        'STATUS_ENCODED': ['mean', 'max', 'first'],
        'WEIGHTED_STATUS': ['mean', 'sum', 'first'],
        'EXP_ENCODED_STATUS': ['last'],
        'EXP_WEIGHTED_STATUS': ['last']
    }
    aggregations_for_year = {
        'STATUS_ENCODED': ['mean', 'max', 'last', 'first'],
        'WEIGHTED_STATUS': ['mean', 'max', 'first', 'last'],
        'EXP_WEIGHTED_STATUS': ['last'],
        'EXP_ENCODED_STATUS': ['last']
    }

    # Aggregate overall
    aggregated_bb = bb.groupby('SK_ID_BUREAU').agg(aggregations_basic)
    aggregated_bb.columns = ['BB_' + '_'.join(col).upper() for col in aggregated_bb.columns]

    # Aggregate yearly (last 2 years + rest)
    aggregated_bb_years = pd.DataFrame(index=aggregated_bb.index) # Start with same index
    for year in range(2):
        year_group = bb[bb['YEAR_BALANCE'] == year].groupby('SK_ID_BUREAU').agg(aggregations_for_year)
        year_group.columns = [f'BB_{"_".join(col).upper()}_YEAR_{year}' for col in year_group.columns]
        aggregated_bb_years = aggregated_bb_years.merge(year_group, on='SK_ID_BUREAU', how='left')

    aggregated_bb_rest_years = bb[bb['YEAR_BALANCE'] >= 2].groupby('SK_ID_BUREAU').agg(aggregations_for_year)
    aggregated_bb_rest_years.columns = [f'BB_{"_".join(col).upper()}_YEAR_REST' for col in aggregated_bb_rest_years.columns]
    aggregated_bb_years = aggregated_bb_years.merge(aggregated_bb_rest_years, on='SK_ID_BUREAU', how='left')

    # Merge yearly aggregates back
    aggregated_bb = aggregated_bb.merge(aggregated_bb_years, on='SK_ID_BUREAU', how='left')

    # Fill NaNs resulting from aggregations (e.g., bureaus with no recent history)
    # Filling with 0 might be okay, but consider if mean/median is better for some fields
    aggregated_bb = aggregated_bb.fillna(0)

    print(f"Finished preprocessing bureau_balance. Initial shape: {initial_shape}, Final aggregated shape: {aggregated_bb.shape}")
    return aggregated_bb

def preprocess_bureau(bureau_df, aggregated_bb):
    """Preprocesses bureau: merges with aggregated bureau_balance, cleans, engineers features,
       aggregates by SK_ID_CURR."""
    print("Preprocessing bureau data...")
    bureau = bureau_df.copy()
    initial_shape = bureau.shape

    # Merge with aggregated bureau_balance
    bureau_merged = bureau.merge(aggregated_bb, on='SK_ID_BUREAU', how='left')
    print(f"Merged bureau with aggregated bureau_balance. Shape: {bureau_merged.shape}")

    # Clean DAYS fields (limiting to past 50 years ~ 18250 days)
    # Use .loc for assignment to avoid warnings
    limit_days = -50 * 365
    cols_to_clean = ['DAYS_CREDIT_ENDDATE', 'DAYS_ENDDATE_FACT', 'DAYS_CREDIT_UPDATE']
    for col in cols_to_clean:
        if col in bureau_merged.columns:
             bureau_merged.loc[bureau_merged[col] < limit_days, col] = np.nan

    # Feature Engineering
    eps = 1e-5
    if 'DAYS_CREDIT' in bureau_merged.columns and 'DAYS_CREDIT_ENDDATE' in bureau_merged.columns:
         bureau_merged['B_CREDIT_DURATION'] = bureau_merged['DAYS_CREDIT_ENDDATE'] - bureau_merged['DAYS_CREDIT'] # End - Start
    if 'CREDIT_DAY_OVERDUE' in bureau_merged.columns:
         bureau_merged['B_FLAG_OVERDUE_RECENT'] = (bureau_merged['CREDIT_DAY_OVERDUE'] > 0).astype(int)
    if 'AMT_CREDIT_MAX_OVERDUE' in bureau_merged.columns and 'B_CREDIT_DURATION' in bureau_merged.columns:
         bureau_merged['B_MAX_AMT_OVERDUE_DURATION_RATIO'] = bureau_merged['AMT_CREDIT_MAX_OVERDUE'] / (bureau_merged['B_CREDIT_DURATION'] + eps)
    if 'AMT_CREDIT_SUM_OVERDUE' in bureau_merged.columns and 'B_CREDIT_DURATION' in bureau_merged.columns:
         bureau_merged['B_CURRENT_AMT_OVERDUE_DURATION_RATIO'] = bureau_merged['AMT_CREDIT_SUM_OVERDUE'] / (bureau_merged['B_CREDIT_DURATION'] + eps)
    if 'AMT_CREDIT_SUM_OVERDUE' in bureau_merged.columns and 'DAYS_CREDIT_ENDDATE' in bureau_merged.columns:
         # Need positive days for ratio denominator - use abs or adjust logic if needed
         bureau_merged['B_AMT_OVERDUE_DURATION_LEFT_RATIO'] = bureau_merged['AMT_CREDIT_SUM_OVERDUE'] / (np.abs(bureau_merged['DAYS_CREDIT_ENDDATE']) + eps)
    if 'CNT_CREDIT_PROLONG' in bureau_merged.columns and 'AMT_CREDIT_MAX_OVERDUE' in bureau_merged.columns:
         bureau_merged['B_CNT_PROLONGED_MAX_OVERDUE_MUL'] = bureau_merged['CNT_CREDIT_PROLONG'] * bureau_merged['AMT_CREDIT_MAX_OVERDUE']
    if 'CNT_CREDIT_PROLONG' in bureau_merged.columns and 'B_CREDIT_DURATION' in bureau_merged.columns:
         bureau_merged['B_CNT_PROLONGED_DURATION_RATIO'] = bureau_merged['CNT_CREDIT_PROLONG'] / (bureau_merged['B_CREDIT_DURATION'] + eps)
    if 'AMT_CREDIT_SUM_DEBT' in bureau_merged.columns and 'AMT_CREDIT_SUM' in bureau_merged.columns:
         bureau_merged['B_CURRENT_DEBT_TO_CREDIT_RATIO'] = bureau_merged['AMT_CREDIT_SUM_DEBT'] / (bureau_merged['AMT_CREDIT_SUM'] + eps)
         bureau_merged['B_CURRENT_CREDIT_DEBT_DIFF'] = bureau_merged['AMT_CREDIT_SUM'] - bureau_merged['AMT_CREDIT_SUM_DEBT']
    if 'AMT_ANNUITY' in bureau_merged.columns and 'AMT_CREDIT_SUM' in bureau_merged.columns:
         bureau_merged['B_AMT_ANNUITY_CREDIT_RATIO'] = bureau_merged['AMT_ANNUITY'] / (bureau_merged['AMT_CREDIT_SUM'] + eps)
    if 'DAYS_CREDIT_UPDATE' in bureau_merged.columns and 'DAYS_CREDIT_ENDDATE' in bureau_merged.columns:
        # Difference in days (both should be negative, difference magnitude matters)
        bureau_merged['B_CREDIT_ENDDATE_UPDATE_DIFF'] = bureau_merged['DAYS_CREDIT_ENDDATE'] - bureau_merged['DAYS_CREDIT_UPDATE']

    # Aggregation by SK_ID_CURR
    print("Aggregating bureau features by SK_ID_CURR...")

    # --- Aggregation Strategy ---
    # 1. Aggregate numeric features overall (mean, sum, max, min, std)
    # 2. One-hot encode categoricals, then aggregate the OHE features (mean gives prevalence)
    # 3. Aggregate features grouped by CREDIT_ACTIVE status (as per original code)

    # 1. Overall Numeric Aggregations
    numeric_cols_bureau = bureau_merged.select_dtypes(include=np.number).columns.tolist()
    # Remove IDs from aggregation list
    numeric_cols_bureau = [col for col in numeric_cols_bureau if col not in ['SK_ID_CURR', 'SK_ID_BUREAU']]

    aggregations_numeric = {}
    for col in numeric_cols_bureau:
         aggregations_numeric[col] = ['mean', 'max', 'min', 'sum', 'std']

    bureau_agg_numeric = bureau_merged.groupby('SK_ID_CURR').agg(aggregations_numeric)
    bureau_agg_numeric.columns = ['B_' + '_'.join(col).upper() + '_OVERALL' for col in bureau_agg_numeric.columns]
    bureau_agg_numeric = bureau_agg_numeric.fillna(0) # Fill NaNs from aggregation (e.g., std dev for single entry)

    # 2. Categorical Aggregations (One-Hot Encoding then Mean)
    categorical_cols_bureau = bureau_merged.select_dtypes(include='object').columns.tolist()
    bureau_for_ohe = bureau_merged[['SK_ID_CURR'] + categorical_cols_bureau].copy()
    bureau_for_ohe = bureau_for_ohe.fillna('MISSING') # Fill NaNs before OHE

    # Using pandas get_dummies for simplicity here
    bureau_ohe = pd.get_dummies(bureau_for_ohe, columns=categorical_cols_bureau, prefix_sep='_')

    # Aggregate OHE features by mean
    bureau_agg_categorical = bureau_ohe.groupby('SK_ID_CURR').mean()
    bureau_agg_categorical.columns = ['B_' + col.upper() + '_MEAN' for col in bureau_agg_categorical.columns] # Prefix columns

    # 3. Aggregations by CREDIT_ACTIVE status
    # Define aggregations for status groups
    aggregations_credit_active = {
        'DAYS_CREDIT': ['mean', 'min', 'max'],
        'CREDIT_DAY_OVERDUE': ['mean', 'max', 'sum'], # Added sum
        'DAYS_CREDIT_ENDDATE': ['mean', 'max', 'min'], # Added min
        'AMT_CREDIT_SUM': ['sum', 'mean', 'max'], # Added mean
        'AMT_CREDIT_SUM_DEBT': ['sum', 'mean'], # Added mean
        'AMT_CREDIT_SUM_OVERDUE': ['max', 'sum', 'mean'], # Added mean
        'AMT_ANNUITY': ['mean', 'sum', 'max'],
        'B_CREDIT_DURATION': ['max', 'mean', 'min', 'std'], # Added min, std
        # Add aggregates from bureau_balance perspective
        'BB_STATUS_ENCODED_MEAN': ['mean', 'max'],
        'BB_WEIGHTED_STATUS_MEAN': ['mean', 'max'],
        'BB_MONTHS_BALANCE_POS_COUNT': ['sum', 'mean'] # How many total months reported per status
    }
    # Filter based on available columns in bureau_merged
    valid_aggs_credit_active = {}
    for col, funcs in aggregations_credit_active.items():
         if col in bureau_merged.columns:
             valid_aggs_credit_active[col] = funcs

    bureau_agg_by_status = pd.DataFrame(index=bureau_agg_numeric.index) # Start with all SK_ID_CURR
    active_statuses = bureau_merged['CREDIT_ACTIVE'].unique()

    for status in active_statuses:
        if pd.isna(status): continue # Skip potential NaN group if exists
        status_safe_name = status.replace(' ', '_').replace('/', '_') # Make status name filename-safe
        group = bureau_merged[bureau_merged['CREDIT_ACTIVE'] == status].groupby('SK_ID_CURR').agg(valid_aggs_credit_active)
        group.columns = [f'B_{"_".join(col).upper()}_CA_{status_safe_name.upper()}' for col in group.columns]
        bureau_agg_by_status = bureau_agg_by_status.merge(group, on='SK_ID_CURR', how='left')

    bureau_agg_by_status = bureau_agg_by_status.fillna(0) # Fill NaNs for customers without certain credit statuses

    # Combine all aggregated parts
    final_bureau_agg = bureau_agg_numeric.merge(bureau_agg_categorical, on='SK_ID_CURR', how='left')
    final_bureau_agg = final_bureau_agg.merge(bureau_agg_by_status, on='SK_ID_CURR', how='left')

    print(f"Finished aggregating bureau data. Initial shape: {initial_shape}, Final aggregated shape: {final_bureau_agg.shape}")
    return final_bureau_agg


In [24]:
def merge_final_data(X_train, X_test, y_train, y_test, bureau_aggregated):
    """Merges the processed application splits with aggregated bureau data."""
    print("Merging aggregated bureau data with application train/test sets...")

    # Make sure X_train and y_train have the same index before combining
    assert len(X_train) == len(y_train), "X_train and y_train must have the same length"
    assert len(X_test) == len(y_test), "X_test and y_test must have the same length"
    
    # Make copies to avoid modifying originals
    train_df = X_train.copy()
    test_df = X_test.copy()
    
    # Ensure indices match before assigning TARGET
    train_df = train_df.reset_index(drop=True)
    y_train_reset = y_train.reset_index(drop=True)
    train_df['TARGET'] = y_train_reset
    
    test_df = test_df.reset_index(drop=True)
    y_test_reset = y_test.reset_index(drop=True)
    test_df['TARGET'] = y_test_reset
    
    # Ensure SK_ID_CURR is available as a column for merging
    if 'SK_ID_CURR' not in train_df.columns:
        print("Error: SK_ID_CURR not in dataframe columns. Cannot proceed with merge.")
        return None, None
    
    # Check for duplicate SK_ID_CURR values before merging
    train_duplicates = train_df['SK_ID_CURR'].duplicated().sum()
    if train_duplicates > 0:
        print(f"Warning: Found {train_duplicates} duplicate SK_ID_CURR values in train_df")
    
    bureau_duplicates = bureau_aggregated['SK_ID_CURR'].duplicated().sum() if 'SK_ID_CURR' in bureau_aggregated.columns else 0
    if bureau_duplicates > 0:
        print(f"Warning: Found {bureau_duplicates} duplicate SK_ID_CURR values in bureau_aggregated")
    
    # Merge using SK_ID_CURR
    train_merged = train_df.merge(bureau_aggregated.reset_index() if bureau_aggregated.index.name == 'SK_ID_CURR' 
                                  else bureau_aggregated, 
                                  on='SK_ID_CURR', how='left')
    test_merged = test_df.merge(bureau_aggregated.reset_index() if bureau_aggregated.index.name == 'SK_ID_CURR' 
                                else bureau_aggregated, 
                                on='SK_ID_CURR', how='left')

    # Verify no TARGET NaNs were introduced during merge
    train_na_count = train_merged['TARGET'].isna().sum()
    if train_na_count > 0:
        print(f"Warning: {train_na_count} NaN values found in TARGET after merge")
    
    # Fill NaNs in bureau columns
    bureau_cols = [col for col in bureau_aggregated.columns if col != 'SK_ID_CURR']
    train_merged[bureau_cols] = train_merged[bureau_cols].fillna(0)
    test_merged[bureau_cols] = test_merged[bureau_cols].fillna(0)

    print(f"Merging finished. Final Train shape: {train_merged.shape}, Final Test shape: {test_merged.shape}")
    return train_merged, test_merged

In [25]:
# --- 5. Main Orchestrator Function ---

def preprocess_credit_data(data_path='../data/raw/', test_size=0.2, random_state=42):
    """Orchestrates the loading, preprocessing, feature engineering, and merging."""
    start_time = datetime.now()
    print(f"Starting Credit Data Preprocessing at: {start_time}")

    # 1. Load Data
    app_train_df, bureau_df, bureau_balance_df = load_initial_data(data_path)

    # 1a. Pre-Split Cleaning (if any absolutely required before split)
    # Handle 'XNA' Gender before splitting, as it removes rows
    if 'CODE_GENDER' in app_train_df.columns:
        initial_rows = app_train_df.shape[0]
        app_train_df = app_train_df[app_train_df['CODE_GENDER'] != 'XNA'].copy()
        rows_removed = initial_rows - app_train_df.shape[0]
        if rows_removed > 0:
            print(f"Removed {rows_removed} rows with 'XNA' gender before splitting.")

    # 2. Split Application Data
    X_train, X_test, y_train, y_test = split_application_data(app_train_df, test_size=test_size, random_state=random_state)

    # 3. Preprocess Application Data (Post-Split)
    X_train = clean_application(X_train)
    X_test = clean_application(X_test)

    X_train, X_test = handle_app_missing_categorical(X_train, X_test) # Fill categorical NaNs

    # Impute EXT_SOURCE - Fit on Train, Transform Train & Test
    X_train, X_test = impute_ext_sources_xgb(X_train, X_test)

    # Numeric Feature Engineering - Apply to Train & Test
    X_train = engineer_app_numeric_features(X_train)
    X_test = engineer_app_numeric_features(X_test)

    # Categorical Interactions - Fit on Train, Transform Train & Test
    X_train, X_test = engineer_app_categorical_interactions(X_train, X_test, y_train)

    # Categorical Encoding (Response Encoding) - Fit on Train, Transform Train & Test
    X_train, X_test = encode_app_categorical_response(X_train, X_test, y_train)

    # 4. Preprocess Bureau & Bureau Balance
    aggregated_bb = preprocess_bureau_balance(bureau_balance_df)
    aggregated_bureau = preprocess_bureau(bureau_df, aggregated_bb)

    # 5. Merge Application and Bureau Data
    train_final, test_final = merge_final_data(X_train, X_test, y_train, y_test, aggregated_bureau)

    # 6. Final Checks
    print("\nFinal NaN check (Train):")
    print(train_final.isnull().sum().sort_values(ascending=False).head(10))
    print("\nFinal NaN check (Test):")
    print(test_final.isnull().sum().sort_values(ascending=False).head(10))
    
    # Fill remaining NaNs with 0 (if any)
    train_final = train_final.fillna(0)
    test_final = test_final.fillna(0)

    end_time = datetime.now()
    print(f"\nPreprocessing finished at: {end_time}")
    print(f"Total time taken: {end_time - start_time}")

    return train_final, test_final

In [26]:
# --- Example Usage ---
if __name__ == "__main__":
    DATA_DIRECTORY = '../data/raw/' # Adjust path as needed
    try:
        final_train_data, final_test_data = preprocess_credit_data(data_path=DATA_DIRECTORY)
        final_train_data.to_csv('../data/processed/train_final.csv', index=False)
        final_test_data.to_csv('../data/processed/test_final.csv', index=False)

    except FileNotFoundError:
        print("\nExecution failed due to missing files. Please check the DATA_DIRECTORY path.")
    except Exception as e:
        print(f"\nAn error occurred during preprocessing: {e}")
        import traceback
        traceback.print_exc()

Starting Credit Data Preprocessing at: 2025-04-05 15:47:34.187197
Loading raw data...
Raw data loaded successfully.
Removed 4 rows with 'XNA' gender before splitting.
Splitting application data (Test size: 0.2, Random State: 42)...
Train shape: (246005, 121), Test shape: (61502, 121)
Applying basic cleaning to DataFrame with shape: (246005, 121)
Dropped low-variance FLAG columns: ['FLAG_DOCUMENT_2', 'FLAG_DOCUMENT_4', 'FLAG_DOCUMENT_10', 'FLAG_DOCUMENT_12', 'FLAG_DOCUMENT_20']
Converted DAYS_BIRTH to years.
Replaced DAYS_EMPLOYED anomaly (365243) with NaN.
Capped OBS_30_CNT_SOCIAL_CIRCLE at 30 (values > 30 set to NaN).
Capped OBS_60_CNT_SOCIAL_CIRCLE at 30 (values > 30 set to NaN).
Converted REGION_RATING_CLIENT to object type.
Converted REGION_RATING_CLIENT_W_CITY to object type.
Basic cleaning finished.
Applying basic cleaning to DataFrame with shape: (61502, 121)
Dropped low-variance FLAG columns: ['FLAG_DOCUMENT_2', 'FLAG_DOCUMENT_4', 'FLAG_DOCUMENT_10', 'FLAG_DOCUMENT_12', 'FLAG_D